# Subscript MySQL Manager
A collection of tools for maintaining the SubScript database hosted on my n-coding.net website.

In [16]:
import mysql.connector
from mysql.connector import errorcode
import configparser
import pandas as pd
import os
import subscript.config as cn
from subscript.custom_funcs import create_database, create_table, locate_database 
import numpy as np

## Configure database connections

In [2]:
config = configparser.ConfigParser()
config.read('../config.ini')
DB_NAME = config.get('connections', 'db_name')
USER_NAME = config.get('connections', 'db_user')
PWD = config.get('connections', 'db_pwd')
HOST_NAME = config.get('connections','db_host')
PORT = config.get('connections', 'db_port')

## Connect to database

In [11]:
def connect():
    cnx = mysql.connector.connect(user = USER_NAME,
    password = PWD, host = HOST_NAME, database = DB_NAME,
    port = PORT)
    cursor = cnx.cursor(buffered = True)
    locate_database(cursor, DB_NAME)
    return cnx, cursor

## Final_time_stats table

#### Create final_time_stats table

In [23]:
csv = pd.read_csv(os.path.join(cn.clean_dir, 'random_forest_time', 'final_time_stats.csv'), dtype = 'unicode')

# Make ID column from player name and realm
csv['id'] = csv.player + '-' + csv.realm

# Get fields from the column names of the csv and replace any '-' for '_' for compatibility
fields = sorted([c.replace('-','_') for c in csv.columns.values if 'Unnamed' not in c], reverse = True)
sizes = ['varchar(100)']*len(csv.columns)

sql = "CREATE TABLE final_time_stats ("
i = 0
for field in fields:
    if i == 0:
        sql = sql + field + ' ' + sizes[i]
    elif '20' not in field:
         sql = sql + ', ' + field + ' ' + sizes[i]
    elif '20' in field: 
        # Fields can't have dashes or start with a number, so add an a for compatibility
        sql = sql + ', ' + 'a' + field + ' ' + sizes[i]
    i = i + 1
sql = sql + ') engine=innodb default charset=utf8;'

print (len(csv.columns))
print(sql)
cnx, cursor = connect()
#cursor.execute(sql)  # Remove '#' if I need to make the table again
cnx.commit()

94
CREATE TABLE final_time_stats (time_since_login varchar(100), status varchar(100), realm varchar(100), player varchar(100), last_login varchar(100), id varchar(100), gear_score varchar(100), engagement varchar(100), a2020_12 varchar(100), a2020_11 varchar(100), a2020_10 varchar(100), a2020_09 varchar(100), a2020_08 varchar(100), a2020_06 varchar(100), a2020_05 varchar(100), a2020_04 varchar(100), a2020_03 varchar(100), a2020_02 varchar(100), a2020_01 varchar(100), a2019_12 varchar(100), a2019_11 varchar(100), a2019_10 varchar(100), a2019_09 varchar(100), a2019_08 varchar(100), a2019_07 varchar(100), a2019_06 varchar(100), a2019_05 varchar(100), a2019_04 varchar(100), a2019_03 varchar(100), a2019_02 varchar(100), a2019_01 varchar(100), a2018_12 varchar(100), a2018_11 varchar(100), a2018_10 varchar(100), a2018_09 varchar(100), a2018_08 varchar(100), a2018_07 varchar(100), a2018_06 varchar(100), a2018_05 varchar(100), a2018_04 varchar(100), a2018_03 varchar(100), a2018_02 varchar(100),

#### Add records to the final_time_stats table

In [49]:
csv = pd.read_csv(os.path.join(cn.clean_dir, 'random_forest_time', 'final_time_stats.csv'), dtype = 'unicode')

# Make ID column from player name and realm
csv['id'] = csv.player + '-' + csv.realm

# Get fields from the column names of the csv and replace any '-' for '_' for compatibility
cols = sorted([c for c in csv.columns.values if 'Unnamed' not in c], reverse = True)
player_fields = [c for c in cols if '20' not in c]
raw_date_fields = [c for c in cols if '20' in c]
date_fields = ['a' + c.replace('-','_') for c in cols if '20' in c]
raw_fields = player_fields + raw_date_fields
sql_fields = player_fields + date_fields
print(len(raw_fields), len(sql_fields))

91 91


#### Insert records into the final_time_stats table

In [94]:
cnx, cursor = connect()
for index, row in csv[153202:].iterrows():
    values =  row[raw_fields]
    values = [str(v) for v in values]
    sql = "INSERT INTO final_time_stats (" + str(sql_fields).replace('[', '').replace(']','').replace("'",'') + ") VALUES (" + '%s,'*(len(sql_fields)-1) +  "%s)"
    cursor.execute(sql, values)
    cnx.commit()

Database haleygee_subscript  is active.


#### Fetch records from the final_time_stats table

In [143]:
cnx, cursor = connect()
sql = "SELECT * FROM final_time_stats WHERE realm = 'whisperwind' AND status = 'risk'"
cursor.execute(sql)
search = cursor.fetchall()

Database haleygee_subscript  is active.


#### Display records from the final_time_stats table

In [144]:
df_search = pd.DataFrame(columns = raw_fields)
for item in search:
    tmp = pd.DataFrame(item)
    tmp = tmp.T
    tmp.columns = raw_fields
    tmp.time_since_login = tmp.time_since_login.str.split('days')[0][0]
    df_search = df_search.append(tmp, ignore_index = True)
df_search.head()

,time_since_login,status,realm,player,last_login,id,gear_score,engagement,2020-12,2020-11,...,2014-10,2014-09,2014-08,2014-07,2014-06,2014-05,2014-04,2014-03,2014-02,2014-01
0,226,risk,whisperwind,portsforsale,2019-10-23,portsforsale-whisperwind,268.0,1.0,0.0,0.0,...,5.0,75.0,3.0,11.0,0.0,16.0,0.0,30.0,9.0,3.0
1,195,risk,whisperwind,tooter,2019-11-23,tooter-whisperwind,386.0,1.0,0.0,0.0,...,1.0,12.0,0.0,0.0,2.0,3.0,5.0,6.0,10.0,3.0
2,46,risk,whisperwind,migflurries,2020-04-20,migflurries-whisperwind,433.0,1.0,0.0,0.0,...,1.0,0.0,7.0,0.0,2.0,4.0,5.0,7.0,7.0,0.0
3,202,risk,whisperwind,sukmehealz,2019-11-16,sukmehealz-whisperwind,384.0,1.0,0.0,0.0,...,2.0,0.0,1.0,0.0,4.0,27.0,3.0,4.0,4.0,0.0
4,41,risk,whisperwind,kaxanna,2020-04-25,kaxanna-whisperwind,401.0,1.0,0.0,0.0,...,3.0,0.0,0.0,1.0,17.0,13.0,2.0,5.0,5.0,0.0


## Create the achievement_list table

In [146]:
csv = pd.read_csv(os.path.join(cn.clean_dir, 'random_forest_features', 'final_curated_recommendations.csv'), dtype = 'unicode')

# Get fields from the column names of the csv and replace any '-' for '_' for compatibility
fields = sorted([c.replace('-','_') for c in csv.columns.values if 'Unnamed' not in c], reverse = True)
sizes = ['varchar(100)']*len(csv.columns)

sql = "CREATE TABLE features ("
i = 0
for field in fields:
    if i == 0:
        sql = sql + field + ' ' + sizes[i]
    else:
         sql = sql + ', ' + field + ' ' + sizes[i]
    i = i + 1
sql = sql + ') engine=innodb default charset=utf8;'

print (len(csv.columns))
print(sql)
cnx, cursor = connect()
cursor.execute(sql)  # Remove '#' if I need to make the table again
cnx.commit()

3
CREATE TABLE features (name varchar(100), importance varchar(100), category varchar(100)) engine=innodb default charset=utf8;
Database haleygee_subscript  is active.


#### Add records to the features table

In [169]:
csv = pd.read_csv(os.path.join(cn.clean_dir, 'random_forest_features', 'final_curated_recommendations.csv'), dtype = 'unicode')

cnx, cursor = connect()
for index, row in csv[6:].iterrows():
    values =  row[fields]
    values = [str(v) for v in values]
    sql = "INSERT INTO features (" + str(fields).replace('[', '').replace(']','').replace("'",'') + ")\
            VALUES (" + '%s,'*(len(fields)-1) +  "%s)"
    cursor.execute(sql, values)
    #print(sql,values)
    cnx.commit()

Database haleygee_subscript  is active.


#### Search records in the features table

In [170]:
cnx, cursor = connect()
sql = "SELECT * FROM features"
cursor.execute(sql)
search = cursor.fetchall()

Database haleygee_subscript  is active.


#### Display records from the features table

In [171]:
df_search = pd.DataFrame(columns = fields)
for item in search:
    tmp = pd.DataFrame(item)
    tmp = tmp.T
    tmp.columns = fields
    df_search = df_search.append(tmp, ignore_index = True)
df_search.head()

,name,importance,category
0,Rustbolt Rebellion,0.03642103754444700,Battle for Azeroth
1,Waveblade Ankoan,0.010034979074682200,Battle for Azeroth
2,The Unshackled,0.007394899241371400,Battle for Azeroth
3,The Heart Forge,0.007120127044205260,Heart of Azeroth
4,Power Up,0.0068742343139909400,Heart of Azeroth
